In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Linear Regression
## d-dimensional case
* Consider $\mathbf{x} \in \mathbb{R}^d$ and $y \in \mathbb{R}$
* Collect a set of $n$ points $\{ (\mathbf{x}_i , y_i ) \}_{i=1}^n$
* Aim: Find the hyperplane of best fit (equivalently line of best fit for $d=1$)
* Consider a function $f(\mathbf{x};\theta)$ which maps $\mathbf{x}_i$ onto $y_i$, parametrised by a set of parameters $\theta$
* We define the parameters $\theta$ to contain two components: $\theta = \{ \mathbf{w} , b \}$, where $\mathbf{w} \in \mathbb{R}^d$ and $b \in \mathbb{R}$, so that:
$$  f(\mathbf{x};\theta) = \mathbf{w}^T \mathbf{x} + b$$

## Solve via least squares
### Closed Form Solution:
* Given a set of points $\{ (\mathbf{x}_i , y_i ) \}_{i=1}^n$, we can determine the parameters $\theta$ via least squares optimisation:
$$ \min_{\theta} \mathcal{L}(\theta) = \min_{\theta} \frac{1}{2} \sum_{i=1}^n \big\{ || f(\mathbf{x};\theta) - y_i ||_2^2 \big\} = \min_{\mathbf{w},b} \frac{1}{2} \sum_{i=1}^n \big\{ || \mathbf{w}^T \mathbf{x} + b - y_i ||_2^2 \big\} $$
* Let $\mathbf{\hat{x}}_i = (1, \mathbf{x}_i) \in \mathbb{R}^{d+1}$ and $W = (b,w_1,\cdots,w_d) \in \mathbb{R}^{d+1}$, and let $\mathbf{X} = (\mathbf{\hat{x_1}}, \cdots, \mathbf{\hat{x_n}})^T \in \mathbb{R}^{n \times d+1}$ and $\mathbf{y} = (y_1, \cdots, y_n)^T \in \mathbb{R}^n$, then the least squares problem has matrix form:
$$ \min_{\theta} \mathcal{L}(\theta) =  \min_{\mathbf{W}} || \mathbf{X} \mathbf{W} - \mathbf{y} ||_2^2 $$

* This has a closed form solution, taken by setting the gradient to $0$:
    * $\nabla_{\mathbf{W}} \mathcal{L}(\theta) = 2 \mathbf{X}^T(\mathbf{X} \mathbf{W} - \mathbf{y})$
    * $ 2 \mathbf{X}^T(\mathbf{X} \mathbf{W} - \mathbf{y}) = 0 \implies \mathbf{W} = \big( \mathbf{X}^T \mathbf{X} \big)^{-1} \mathbf{X}^T \mathbf{y} $

* In the following example we solve linear regression using numpy for a simple example where $d=1$ (i.e. $f(x;\theta) = wx + b$, $w,b \in \mathbb{R}$)


In [ ]:
# Solving the least-square problem in numpy
# Fit a line, ``y = wx + b``, through some noisy data-points: 
x = np.array([0, 1, 2, 3])
y = np.array([-1, 0.2, 0.9, 2.1])

A = np.vstack([x, np.ones(len(x))]).T # vertical stack followed by a transposition. ( y = Ap, A=[[x 1]] and p =[[w],[b]] )
print("The data matrix: \n", A) 
#numpy as a built in function achieving least squares
w_np, b_np = np.linalg.lstsq(A, y, rcond=None)[0]
print("The obtained result: ", w_np, b_np)

In [ ]:
# Plotting the line of best fit

import matplotlib.pyplot as plt
plt.plot(x, y, 'o', label='Original data', markersize=10)
plt.plot(x, w_np*x + b_np, 'r', label='Fitted line')
plt.legend()

### Gradient Descent
* It is often the case that a closed form solution is too computationally expensive. Gradient descent is a widely used alternative.
* Start with some initial parameters $\theta^{(0)}$ with some (small) time-step $\tau$, gradient descent updates $\theta^{(k)}$ iteratively by the following update:

$$ \theta^{(k+1)} = \theta^{(k)} - \tau \nabla_{\theta} \mathcal{L}(\theta^{(k)}$$

For the following least squares problem ($\mathbf{x},\mathbf{w} \in \mathbb{R}^d, y \in \mathbb{R})$:
$$ \min_{\theta} \mathcal{L}(\theta) = \min_{\theta} \sum_{i=1}^n \big\{ || f(\mathbf{x};\theta) - y_i ||_2^2 \big\} = \min_{\mathbf{w},b} \sum_{i=1}^n \big\{ || \mathbf{w}^T \mathbf{x} + b - y_i ||_2^2 \big\} $$
We have the following gradients:
$$ \nabla_{\mathbf{w}} \mathcal{L}(\theta) = \sum_{i=1}^n \mathbf{x}_i ( \mathbf{w}^T \mathbf{x}_i + b - y_i) $$
$$ \nabla_{b} \mathcal{L}(\theta) = \sum_{i=1}^n ( \mathbf{w}^T \mathbf{x}_i + b - y_i) $$
And so:
$$ \mathbf{w}^{(k+1)} = \mathbf{w}^{(k)} - \tau \sum_{i=1}^n \mathbf{x}_i ( \mathbf{w^{(k)}}^T \mathbf{x}_i + b^{(k)} - y_i) $$
$$ b^{(k+1)} = b^{(k)} - \tau \sum_{i=1}^n  ( \mathbf{w^{(k)}}^T \mathbf{x}_i + b^{(k)} - y_i) $$

In [ ]:
#Solving least squares using gradient descent

# Fit a line, ``y = wx + b``, through some noisy data-points, where d=1.
x = np.array([0, 1, 2, 3])
y = np.array([-1, 0.2, 0.9, 2.1])

def fit_lse(x, y):
    lr = 0.1 # the learning rate
    w = 1
    b = 0
    for i in range(10):
        # initializing the gradiant w.r.t w and b
        grad_w = 0. 
        grad_b = 0. 
        
        # Going through the entire dataset
        for _x, _y in zip(x, y):
            # accumulating the gradient w.r.t w and b
            grad_w += _x*(_x*w + b - _y) 
            grad_b += (_x*w + b - _y)
        
        # Gradient descent steps
        w = w - lr*(grad_w)
        b = b - lr*(grad_b)
        
    return w, b

w, b = fit_lse(x, y)
print("The obtained result: ", w, b)

In [ ]:
# Plotting the line of best fit

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(x, y, 'o', label='Original data', markersize=10)
plt.plot(x, w_np*x + b_np, 'r', label='Fitted line')
plt.title("Solved using numpy function")
plt.legend()
plt.subplot(1,2,2)
plt.plot(x, y, 'o', label='Original data', markersize=10)
plt.plot(x, w*x + b, 'r', label='Fitted line')
plt.title("Solved using gradient descent")
plt.legend()
plt.show()

## Higher order polynomial regression
* Linear functions sometimes don't capture the complexity of data, we can instead use higher order polynomials: $$f(\mathbf{x}; \theta) = b + w_1 \mathbf{x} + w_2 \mathbf{x}^2 + w_3 \mathbf{x}^3 + \cdots .$$
* Consider another example using quadratics. For simplicity let $d=1$ so that $f(x,\theta) = b + w_1 x + w_2 x^2$, $\theta = \{ b , w_1, w_2 \}$. We have:
$$ \min_{\theta} \mathcal{L}(\theta) = \min_{b, w_1, w_2} \sum_{i=1}^n || b + w_1 x_i + w_2 x_i^2 - y_i ||_2^2 $$
So that:
$$ \nabla_b \mathcal{L}(\theta) = \sum_{i=1}^n   ( b + w_1 x_i + w_2 x_i^2 - y_i) $$
$$ \nabla_{w_1} \mathcal{L}(\theta) = \sum_{i=1}^n  x_i ( b + w_1 x_i + w_2 x_i^2 - y_i) $$
$$ \nabla_{w_2} \mathcal{L}(\theta) = \sum_{i=1}^n  x_i^2 ( b + w_1 x_i + w_2 x_i^2 - y_i) $$

$$ b^{(k+1)} = b^{(k)} - \tau \nabla_{b} \mathcal{L}(b^{(k)}$$
$$ w_1^{(k+1)} = w_1^{(k)} - \tau \nabla_{w_1} \mathcal{L}(w_1^{(k)}$$
$$ w_2^{(k+1)} = w_2^{(k)} - \tau \nabla_{w_2} \mathcal{L}(w_2^{(k)}$$

## Homework:
Complete the following function to do gradient descent applied to quadratic regression

In [ ]:
#Homework:
import numpy as np
import matplotlib.pyplot as plt

x = np.array([0.0, 1.0, 2.0, 3.0,  4.0,  5.0])
y = np.array([0.0, 0.8, 0.9, 0.1, -0.8, -1.0])
plt.plot(x, y, 'o', label='Original data', markersize=10)

def fit_lse_quadratic(x, y):
    lr = 0.001 # the learning rate, 'tau'

    
    
    
    
    
    
    
    
    
        
    return b, w1, w2

b, w1, w2 = fit_lse_quadratic(x, y)
print("The obtained result: ", b, w1, w2)

In [ ]:
# Plotting 
plt.plot(x, y, 'o', label='Original data', markersize=10)
plt.plot(x, w2*np.square(x)+ w1*x + b, 'r', label='Fitted line')
plt.legend()

# Binary Logistic Regression

* Suppose we have a set of points $\big\{ ( \mathbf{x}_i \in \mathbb{R}^d, y_i \in \{ 0, 1 \} ) \big\}_{i=1}^n$
* The aim is to build a classification model. We define a function $f$ parameterised by parameters $\theta = \{\mathbf{w}, b\}$, similar to linear regression, but this time based on a sigmoid function.
$$ f(\mathbf{x}) = \frac{1}{1 + \exp(-( \mathbf{x}^T \mathbf{w} + b))} $$
Then is $y$ is found by thresholding at $\frac{1}{2}$:
$$ y = \begin{cases}
1, \text{ if } f (\mathbf{x} ; \theta ) \geq \frac{1}{2} \\
0, \text{ else}
\end{cases} $$

In [ ]:
#Illustration of the sigmoid function
x = np.linspace(-10,10,100)
sigmoid = 1./(1. + np.exp(-x))


plt.plot(x, sigmoid)
plt.legend(["The Sigmoid function"])
plt.show()

### Log-Likelihood as loss function
* We use a probabilistic interpreation of the sigmoid function (Bernoulli distribution):
$$ p(y=1 | \mathbf{x} ; \theta ) = f(\mathbf{x};\theta),$$
$$ p(y=0 | \mathbf{x} ; \theta ) = 1- f(\mathbf{x};\theta),$$
* The likelihood, $L(\theta)$ is defined as:
\begin{align} L(\theta) = p(y|\mathbf{x};\theta) &= f(\mathbf{x};\theta)^y ( 1 - f(\mathbf{x};\theta))^y \\
&= \prod_{i=1}^n f(\mathbf{x}_i;\theta)^{y_i} ( 1 - f(\mathbf{x}_i;\theta))^{y_i} \end{align}
* We now define the log-likelihood as:
\begin{align} \log L(\theta) &=  \sum_{i=1}^n y_i \log f(\mathbf{x_i} ; \theta) + (1-y_i) \log ( 1 - f(\mathbf{x_i};\theta)) \end{align}

* We now convert this log-likelihood into a convex function that we can, then, minimize to update our parameters.

* The standard logistic regression loss function is given by the negative log-likelihood (a.k.a the cross-entropy loss): 
$$ \mathcal{L}(\theta) = -\log L(\theta) $$

* Next we perform gradient descent to update the parameters. The gradients are as follows:
\begin{align}
\nabla_{\mathbf{w}} \mathcal{L}(\theta) &= - \sum_{i=1}^n y_i \nabla_{\mathbf{w}} \log f(\mathbf{x_i}; \theta) + (1-y_i) \nabla_{\mathbf{w}} \log (1-f(\mathbf{x_i}; \theta) ), \\
&= - \sum_{i=1}^n \Big( y_i \frac{1}{f(\mathbf{x_i}; \theta)} + (1-y_i) \frac{-1}{1 - f(\mathbf{x_i}; \theta)} \Big)    \nabla_{\mathbf{w}} f(\mathbf{x_i}; \theta), \\
&= - \sum_{i=1}^n \Big( y_i \frac{1}{f(\mathbf{x_i}; \theta)} + (1-y_i) \frac{-1}{1 - f(\mathbf{x_i}; \theta)} \Big)    f(\mathbf{x_i}; \theta) (1 - f(\mathbf{x_i}; \theta)) \nabla_{\mathbf{w}}(\mathbf{x_i}^T \mathbf{w} + b), \\
&= - \sum_{i=1}^n \Big( y_i \frac{1}{f(\mathbf{x_i}; \theta)} + (1-y_i) \frac{-1}{1 - f(\mathbf{x_i}; \theta)} \Big)    f(\mathbf{x_i}; \theta) (1 - f(\mathbf{x_i}; \theta)) \mathbf{x_i}, \\
& = - \sum_{i=1}^n \big( y_i - f(\mathbf{x_i};\theta) \big) \mathbf{x_i}
\end{align}

\begin{align}
\nabla_{b} \mathcal{L}(\theta)  &= - \sum_{i=1}^n \Big( y_i \frac{1}{f(\mathbf{x_i}; \theta)} + (1-y_i) \frac{-1}{1 - f(\mathbf{x_i}; \theta)} \Big)    f(\mathbf{x_i}; \theta) (1 - f(\mathbf{x_i}; \theta)) \nabla_{b}(\mathbf{x_i}^T \mathbf{w} + b), \\
&= - \sum_{i=1}^n \Big( y_i \frac{1}{f(\mathbf{x_i}; \theta)} + (1-y_i) \frac{-1}{1 - f(\mathbf{x_i}; \theta)} \Big)    f(\mathbf{x_i}; \theta) (1 - f(\mathbf{x_i}; \theta)) , \\
& = - \sum_{i=1}^n \big( y_i - f(\mathbf{x_i};\theta) \big)
\end{align}

## Implementing a logistic regressor:

In [ ]:
import numpy as np

class MyBinaryLogisticRegression:
    def __init__(self, n_iter, lr):
        """
        A class for binary logistic regression model
         
        n_iter: (integer)
            Number of training iteration
        lr: (float)
            learning rate
        """

        self.n_iter = n_iter
        self.lr = lr
        
        self.losses = []
        self.grads = []

    def init_params(self, n_feats):
        self.w = np.zeros((1, n_feats))
        self.b = 0.
        
    def _optimize(self, X, y):
        m = X.shape[0]

        # cost function
        probs = self.activation(X) 
        cost = (-1/m)*(np.sum((y*np.log(probs)) + ((1-y)*(np.log(1-probs)))))

        # computing the gradient 
        dw = (1/m)*(np.dot((probs - y), X)) 
        

            
        db = (1/m)*(np.sum(probs - y))

        grads = {"dLdw": dw, "dLdb": db}

        return grads, cost

    def activation(self, X):  
        """
            Compute the Sigmoid activation on the dataset
            
            X: (np-array) shape=(n_samples x n_feats)
                data matrix 
        """          
        return 1./(1. + np.exp(-np.dot(self.w, X.T) - self.b))
    
    def fit(self, X, y):
        """
            Training the model
            
            X: (np-array) shape=(n_samples x n_feats)
                data matrix
            y: (np-array) shape=( n_samples)
                targets 
        """
        
        self.init_params(X.shape[1])
        
        for i in range(self.n_iter):
            grads, cost = self._optimize(X, y)
            #
            dLdw = grads['dLdw']
            dLdb = grads['dLdb']
            
            # gradient descent
            self.w = self.w - self.lr * dLdw
            self.b = self.b - self.lr * dLdb
            
            self.losses.append(cost)
            self.grads.append(grads)
            #if (i % 10 == 0):
            #    print("Standard Logistic Regression: Iter {}, Cost {}".format(i, cost))
            
        print("Standard Logistic Regression: Iter {}, Cost {}".format(i, cost))
    
    def predict(self, X):
        """
            Predicting the discrete labels
            
            X: (np-array) shape=(n_samples x n_feats)
                data matrix 
        """
        activ = self.activation(X)
        return activ >= 0.5 
    
    def predict_proba(self, X):
        
        """
            Predicting the probabilities

            X: (np-array) shape=(n_samples x n_feats)
                data matrix 
        """
        return self.activation(X) 
    
    def score(self, X, y): 
        """
            computing the accuracy of the model
            
            X: (np-array) shape=(n_samples x n_feats)
                data matrix 
        """
        pred = self.predict(X)
        return (pred == y).mean()

In [ ]:
# To Try out the model, we start by generating a random binary dataset

import numpy as np
import matplotlib.pyplot as plt 
from sklearn.datasets import make_classification

# we create 200 separable points
X, y = make_classification(n_samples=200, n_features=2, n_redundant=0, n_informative=2,
                           random_state=0, n_clusters_per_class=2, class_sep=2.0)
plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='black', s=25) 
plt.title('Plot of the generated data and their label')
plt.show()

In [ ]:
# Instanciating the model and training
my_clf = MyBinaryLogisticRegression(n_iter=100, lr=0.1)
my_clf.fit(X, y)

In [ ]:
# Plotting the decision boundary of our Binary Logistic Regressor
def print_decision(X, y, clf, title="Decision Boundary"):
    plt.figure()
    h = .02
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    ax = plt.subplot(111)
    ax.contourf(xx, yy, Z, alpha=.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='black', s=25) 
    plt.title(title)

In [ ]:
print_decision(X, y,  my_clf, "Decision Boundary of our Binary Logistic Regressor")

In [ ]:
cost = my_clf.losses
plt.plot(cost)

## Recommended resources
- [3Blue1Brown's Neural Network Series](https://www.youtube.com/watch?v=aircAruvnKk&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi)